In [1]:
"""
--- Day 11: Cosmic Expansion ---

You continue following signs for "Hot Springs" and eventually come across an observatory. The Elf within turns 
out to be a researcher studying cosmic expansion using the giant telescope here.

He doesn't know anything about the missing machine parts; he's only visiting for this research project. However, 
he confirms that the hot springs are the next-closest area likely to have people; he'll even take you straight 
there once he's done with today's observation analysis.

Maybe you can help him with the analysis to speed things up?

The researcher has collected a bunch of data and compiled the data into a single giant image (your puzzle input). 
The image includes empty space (.) and galaxies (#). For example:
    ...#......
    .......#..
    #.........
    ..........
    ......#...
    .#........
    .........#
    ..........
    .......#..
    #...#.....
The researcher is trying to figure out the sum of the lengths of the shortest path between every pair of 
galaxies. However, there's a catch: the universe expanded in the time it took the light from those galaxies to 
reach the observatory.

Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows 
or columns that contain no galaxies should all actually be twice as big.

In the above example, three columns and two rows contain no galaxies.
These rows and columns need to be twice as big; the result of cosmic expansion therefore looks like this (->):
       v  v  v          ....#........
     ...#......         .........#...
     .......#..         #............
     #.........         .............
    >..........<        .............
     ......#...    ->   ........#....
     .#........         .#...........
     .........#         ............#
    >..........<        .............
     .......#..         .............
     #...#.....         .........#...
       ^  ^  ^          #....#.......
Equipped with this expanded universe, the shortest path between every pair of galaxies can be found. It can help 
to assign every galaxy a unique number:
    ....1........
    .........2...
    3............
    .............
    .............
    ........4....
    .5...........
    ............6
    .............
    .............
    .........7...
    8....9.......
In these 9 galaxies, there are 36 pairs. Only count each pair once; order within the pair doesn't matter. For 
each pair, find any shortest path between the two galaxies using only steps that move up, down, left, or right 
exactly one . or # at a time. (The shortest path between two galaxies is allowed to pass through another galaxy.)

For example, here is one of the shortest paths between galaxies 5 and 9:
    ....1........
    .........2...
    3............
    .............
    .............
    ........4....
    .5...........
    .##.........6
    ..##.........
    ...##........
    ....##...7...
    8....9.......
This path has length 9 because it takes a minimum of nine steps to get from galaxy 5 to galaxy 9 (the eight 
locations marked # plus the step onto galaxy 9 itself). Here are some other example shortest path lengths:
 - Between galaxy 1 and galaxy 7: 15
 - Between galaxy 3 and galaxy 6: 17
 - Between galaxy 8 and galaxy 9: 5
Here, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is 374.

Expand the universe, then find the length of the shortest path between every pair of galaxies. 
What is the sum of these lengths?

"""

import numpy as np

from itertools import combinations

def expand(arr: np.ndarray) -> np.ndarray:
    new_columns = []
    for c in range(arr.shape[1]):
        new_columns.append(arr[:, c])
        if '#' not in arr[:, c]:
            new_columns.append(arr[:, c])
    arr = np.column_stack(new_columns)
    new_rows = []
    for r in range(arr.shape[0]):
        new_rows.append(arr[r])
        if '#' not in arr[r]:
            new_rows.append(arr[r])
    return np.row_stack(new_rows)
    
with open("input.txt") as f:
    txt = [*map(str.strip, f.readlines())]
    shape = len(txt), len(txt[0])

grid = np.array([*map(list,txt)]).reshape(shape)
grid = expand(grid)
galaxies = [tuple(i) for i in np.argwhere(grid == '#')]

print(sum(abs(s[0] - e[0]) + abs(s[1] - e[1]) for s, e in combinations(galaxies, 2)))
            
# sum is 10490062

10490062


In [2]:
"""
--- Part Two ---

The galaxies are much older (and thus much farther apart) than the researcher initially estimated.

Now, instead of the expansion you did before, make each empty row or column one million times larger. That is, 
each empty row should be replaced with 1000000 empty rows, and each empty column should be replaced with 1000000 
empty columns.

(In the example above, if each empty row or column were merely 10 times larger, the sum of the shortest paths 
between every pair of galaxies would be 1030. If each empty row or column were merely 100 times larger, the sum 
of the shortest paths between every pair of galaxies would be 8410. However, your universe will need to expand 
far beyond these values.)

Starting with the same initial image, expand the universe according to these new rules, then find the length of 
the shortest path between every pair of galaxies. What is the sum of these lengths?

"""

import numpy as np

from itertools import combinations
from typing import Tuple

def get_dist(exp_c: int, exp_r: int, s: Tuple[int], e: Tuple[int]) -> int:
    multiplier = 1000000
    sy, sx = s
    ey, ex = e
    base_x = abs(sx - ex)
    base_y = abs(sy - ey)
    a = sum([exp in range(*sorted([sy, ey])) for exp in exp_r]) * (multiplier - 1)
    b = sum([exp in range(*sorted([sx, ex])) for exp in exp_c]) * (multiplier - 1)
    return base_x + base_y + a + b
    
with open("input.txt") as f:
    txt = [*map(str.strip, f.readlines())]
    shape = len(txt), len(txt[0])
    
grid = np.array([*map(list,txt)]).reshape(shape)
galaxies = [tuple(i) for i in np.argwhere(grid == '#')]
exp_r = np.where(~np.any(grid == '#', axis=1))[0]
exp_c = np.where(~np.any(grid == '#', axis=0))[0]

print(sum(get_dist(exp_c, exp_r, s, e) for s, e in combinations(galaxies, 2)))
            
# sum is 382979724122

382979724122
